In [1]:
import re
import json
import pandas as pd
from google.cloud import bigquery

In [2]:
%load_ext google.cloud.bigquery

In [56]:
%%bigquery
SELECT
    count(*)
FROM `hde-test-clean.housing_data.real_estate_raw`

,f0_
0,1245


In [4]:
query_client = bigquery.Client()

In [5]:
request = """
SELECT
    *
FROM
    `hde-test-clean.housing_data.real_estate_raw`
"""
real_estate_raw = query_client.query(request).to_dataframe()

In [6]:
real_estate_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245 entries, 0 to 1244
Data columns (total 11 columns):
ad_id                1245 non-null int64
ad_url               1245 non-null object
new_building         1245 non-null bool
short_description    1245 non-null object
full_description     1245 non-null object
property_table       1245 non-null object
main_price           1245 non-null object
property_attrs       1245 non-null object
address              1245 non-null object
datetime_viewed      1245 non-null datetime64[ns]
datetime_offset      1245 non-null object
dtypes: bool(1), datetime64[ns](1), int64(1), object(8)
memory usage: 98.6+ KB


In [7]:
real_estate_raw.set_index(keys='ad_id', drop=True, inplace=True)

In [105]:
real_estate_raw.head()

,ad_url,new_building,short_description,full_description,property_table,main_price,property_attrs,address,datetime_viewed,datetime_offset
ad_id,,,,,,,,,,
149269674,https://www.finn.no/realestate/homes/ad.html?f...,False,Ny og delikat 3-roms selveierleil. midt i hjer...,Velkommen til Kantorveien 4 B - moderne og fun...,{},7 850 000 kr,"{'Omkostninger': '27 042 kr', 'Totalpris': '7 ...","Kantorveien 4B, 1410 Kolbotn",2019-06-11 12:38:54,UTC
149797548,https://www.finn.no/realestate/homes/ad.html?f...,False,Arealeffektiv og sentral 2-roms selveier leili...,Aktiv Eiendomsmegling ved Celine B. S. Holm ha...,{},1 690 000 kr,"{'Fellesgjeld': '32 000 kr', 'Omkostninger': '...","Herkulesvegen 77D, 2165 Hvam",2019-06-11 12:41:27,UTC
149814042,https://www.finn.no/realestate/homes/ad.html?f...,False,Gjennomgående 3-roms med balkong l Idyllisk ut...,Velkommen til Ammerudveien 51! Leiligheten lig...,{},2 600 000 kr,"{'Fellesgjeld': '172 000 kr', 'Omkostninger': ...","Ammerudveien 51, 0958 Oslo",2019-06-11 12:40:53,UTC
149828158,https://www.finn.no/realestate/homes/ad.html?f...,False,Stor og vakker leilighet - to balkonger - stor...,Velkommen til denne fantastiske leiligheten!De...,{},8 000 000 kr,"{'Fellesgjeld': '99 566 kr', 'Omkostninger': '...","Waldemar Thranes gate 66 D, 0173 Oslo",2019-06-11 12:40:32,UTC
147199162,https://www.finn.no/realestate/homes/ad.html?f...,False,Unikt renoveringsobjekt. 7-roms ene- el. 2-man...,Huset har til nå fylt rollen som generasjonsbo...,{},8 700 000 kr,"{'Omkostninger': '229 822 kr', 'Totalpris': '8...","Lalienveien 23, 1453 Bjørnemyr",2019-06-11 12:39:13,UTC


In [9]:
for col in ['property_table', 'property_attrs']:
    real_estate_raw[col] = [json.loads(cell) for cell in real_estate_raw[col]]

In [60]:
headers = []
for id, row in real_estate_raw.property_attrs.items():
    for k in row.keys():
        headers.append(k)

In [99]:
headers = list(set(headers))

In [63]:
all_attributes_map = {'Låneverdi': 'loan_value',
                      'Soverom': 'num_bedrooms',
                      'Primærrom': 'primary_size',
                      'Firma': 'company',
                      'Eieform': 'ownership_type',
                      'Renovert år': 'renovation_year',
                      'Bruksareal': 'usable_area',
                      'Mobil': 'mobile',
                      'Fellesgjeld': 'common_debt',
                      'Tomt': 'plot',
                      'Lånetakst': 'mortgage_tariff',
                      'Tomteareal': 'plot_size',
                      'Boligtype': 'property_type',
                      'Referanse': 'reference',
                      'Bruttoareal': 'total_size',
                      'Totalpris': 'price',
                      'Areal': 'size',
                      'FINN-kode': 'ad_id',
                      'Verditakst': 'value_tariff',
                      'Eierskifte-forsikring': 'insurance_change_of_ownership',
                      'Felleskost/mnd.': 'common_expenses_per_month',
                      'Ferdigstillelse': 'completion',
                      'Kommunale avg.': 'municipality_taxes',
                      'Pris med fellesgjeld': 'price_incl_common_debt',
                      'Solforhold': 'sun_light',
                      'Sist endret': 'last_changed',
                      'Sikringsordning': 'security_arrangement',
                      'Energimerking': 'energy_grade',
                      'Fellesformue': 'common_wealth',
                      'Telefon': 'phone',
                      'Enhetsid': 'unit_id',
                      'Grunnflate': 'surface_area',
                      'Omkostninger': 'brokerage_expenses',
                      'Festeavgift': 'reservation_rent',
                      'Formuesverdi': 'wealth_value',
                      'Utendørsareal': 'outdoors_area',
                      'Rom': 'num_rooms',
                      'Ant etasjer': 'num_floors',
                      'Felleskost/mnd. etter avdragsfri periode': 'common_debt_after_no_fee_period',
                      'Byggeår': 'construction_year',
                      'Fax': 'fax',
                      'Etasje': 'floor'}

In [100]:
to_append = []
for code, row in real_estate_raw.property_attrs.items():
    all_properties = []
    for key in headers:
        all_properties.append(row.get(key))
    to_append.append(all_properties)
building_attributes = pd.DataFrame(to_append, columns=headers)
building_attributes.rename(columns=all_attributes_map, inplace=True)

In [107]:
def get_int_from_str(string):
    if string:
        try:
            concat_string = ''.join(re.findall(r'[\d.]', string))
            if concat_string != '':
                return float(concat_string)
        except Exception as e:
            print(e, string)
            pass
    else:
        return None

In [108]:
building_attributes_parsed = building_attributes.copy()
for col in ['loan_value', 'num_bedrooms', 'primary_size',
            'renovation_year', 'usable_area', 'common_debt',
            'plot', 'mortgage_tariff', 'plot_size',
            'total_size', 'price', 'size', 'value_tariff',
            'common_expenses_per_month', 'municipality_taxes',
            'price_incl_common_debt', 'common_wealth',
            'surface_area', 'brokerage_expenses', 'reservation_rent',
            'wealth_value', 'outdoors_area','num_rooms',
            'num_floors', 'common_debt_after_no_fee_period',
            'construction_year', 'floor']:
    building_attributes_parsed[col] = [get_int_from_str(s) for s in building_attributes_parsed[col]]

In [109]:
building_attributes_parsed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1245 entries, 0 to 1244
Data columns (total 42 columns):
loan_value                         2 non-null float64
num_bedrooms                       1206 non-null float64
primary_size                       1159 non-null float64
company                            6 non-null object
ownership_type                     1245 non-null object
renovation_year                    17 non-null float64
usable_area                        1159 non-null float64
mobile                             226 non-null object
common_debt                        624 non-null float64
plot                               0 non-null object
mortgage_tariff                    8 non-null float64
plot_size                          1115 non-null float64
property_type                      1245 non-null object
reference                          1223 non-null object
total_size                         1043 non-null float64
price                              1141 non-null float64
siz

In [48]:
sub_properties_parsed.drop_duplicates(subset=['apt_id', 'num_bedrooms', 'floor', 'primary_size', 'total_size', 'price'], inplace=True)

In [49]:
sub_properties_parsed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 613 entries, 0 to 1088
Data columns (total 7 columns):
apt_id          613 non-null object
num_bedrooms    611 non-null float64
floor           538 non-null float64
primary_size    469 non-null float64
total_size      612 non-null float64
price           610 non-null float64
ad_id           613 non-null float64
dtypes: float64(6), object(1)
memory usage: 38.3+ KB


In [58]:
expanded_sub_properties_schema = [{'name': 'apt_id', 'type': 'STRING'},
                                  {'name': 'num_bedrooms', 'type': 'INTEGER'},
                                  {'name': 'floor', 'type': 'INTEGER'},
                                  {'name': 'primary_size', 'type': 'FLOAT'},
                                  {'name': 'total_size', 'type': 'FLOAT'},
                                  {'name': 'price', 'type': 'FLOAT'},
                                  {'name': 'ad_id', 'type': 'INTEGER'},]

In [59]:
sub_properties_parsed.to_gbq(destination_table='housing_data.expanded_sub_properties',
                             project_id='hde-test-clean',
                             table_schema=expanded_sub_properties_schema,
                             if_exists='append')